# 3.7 Nonlinear problems
We want to solve a nonlinear PDE. 

## A simple scalar PDE
We consider the simple PDE 

$$
- \Delta u + 3 u^3 = 1 \text{ in } \Omega
$$

on the unit square $\Omega = (0,1)^2$. 

We note that this PDE can also be formulated as a nonlinear minimization problem (cf. [3.8](../unit-3.8-nonlmin/nonlmin.ipynb)).

In [11]:
import ngsolve as ngs
import numpy as np
from netgen.geom2d import unit_square
from ngsolve.webgui import Draw
mesh = ngs.Mesh (unit_square.GenerateMesh(maxh=0.3))

In NGSolve we can solve the PDE conveniently using the *linearization* feature of `SymbolicBFI`.

The `BilinearForm` (which is not bilinear!) needed in the weak formulation is
$$
  A(u,v) = \int_{\Omega} \nabla u \nabla v + 3 u^3 v - 1 v ~ dx \quad ( = 0 ~ \forall~v \in H^1_0)
$$

In [7]:
V = ngs.H1(mesh, order=3, dirichlet=[1,2,3,4])
u,v = V.TnT()
a = ngs.BilinearForm(V)
a += (ngs.grad(u) * ngs.grad(v) + 3*u**3*v- 1 * v)*ngs.dx

### Newton's method

We use Newton's method and make the loop:

* Given an initial guess $u^0$
* loop over $i=0,..$ until convergence:
  * Compute linearization: $A u^i + \delta A(u^i) \Delta u^{i} = 0$:
    * $f^i = A u^i$ 
    * $B^i = \delta A(u^i)$ 
    * Solve $B^i \Delta u^i = -f^i$
  * Update $u^{i+1} = u^i + \Delta u^{i}$
  * Evaluate stopping criteria

As a stopping criteria we take $\langle A u^i,\Delta u^i \rangle = \langle A u^i, A u^i \rangle_{(B^i)^{-1}}< \varepsilon$.  

In [14]:
def SimpleNewtonSolve(gfu,a,tol=1e-13,maxits=25):
    res = gfu.vec.CreateVector()
    du = gfu.vec.CreateVector()
    fes = gfu.space
    for it in range(maxits):
        print ("Iteration {:3}  ".format(it),end="")
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        du.data = a.mat.Inverse(fes.FreeDofs()) * res
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = np.sqrt(np.abs(ngs.InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < tol:
            break

In [15]:
gfu = ngs.GridFunction(V)
Draw(gfu,mesh,"u")
SimpleNewtonSolve(gfu,a)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2009', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': F…

Iteration   0  <A u 0 , A u 0 >_{-1}^0.5 =  0.18743532107668498
Iteration   1  <A u 1 , A u 1 >_{-1}^0.5 =  9.416472885494966e-05
Iteration   2  <A u 2 , A u 2 >_{-1}^0.5 =  8.538474295647304e-11
Iteration   3  <A u 3 , A u 3 >_{-1}^0.5 =  4.174306205888498e-17


There are also some solvers shipped with NGSolve now:

In [16]:
from ngsolve.solvers import *
help(Newton)

Help on function Newton in module ngsolve.nonlinearsolvers:

Newton(a, u, freedofs=None, maxit=100, maxerr=1e-11, inverse='umfpack', dirichletvalues=None, dampfactor=1, printing=True, callback=None)
    Newton's method for solving non-linear problems of the form A(u)=0.
    
    Parameters
    ----------
    a : BilinearForm
      The BilinearForm of the non-linear variational problem. It does not have to be assembled.
    
    u : GridFunction
      The GridFunction where the solution is saved. The values are used as initial guess for Newton's method.
    
    freedofs : BitArray
      The FreeDofs on which the assembled matrix is inverted. If argument is 'None' then the FreeDofs of the underlying FESpace is used.
    
    maxit : int
      Number of maximal iteration for Newton. If the maximal number is reached before the maximal error Newton might no converge and a warning is displayed.
    
    maxerr : float
      The maximal error which Newton should reach before it stops. The er

In [17]:
gfu.vec[:]=0
Newton(a,gfu,freedofs=gfu.space.FreeDofs(),maxit=100,maxerr=1e-11,inverse="umfpack",dampfactor=1,printing=True)

Newton iteration  0
err =  0.18743532107668498
Newton iteration  1
err =  9.416472885495649e-05
Newton iteration  2
err =  8.538473774632396e-11
Newton iteration  3
err =  3.9599919999714353e-17


(0, 4)

## A trivial problem:

$$
  5 u^2 = 1, \qquad u \in \mathbb{R}.
$$

In [ ]:
V = NumberSpace(mesh)
u,v = V.TnT()
a = BilinearForm(V)
a += ( 5*u*u*v - 1 * v)*dx
gfu = GridFunction(V)
gfu.vec[:] = 1
SimpleNewtonSolve(gfu,a)

print("\nscalar solution", gfu.vec[0], "(exact: ", sqrt(0.2), ")")

## Another example: Stationary Navier-Stokes:
Find $\mathbf{u} \in \mathbf{V}$, $p \in Q$, $\lambda \in \mathbb{R}$ so that
\begin{align}
\int_{\Omega} \nu \nabla \mathbf{u} : \nabla \mathbf{v} + (\mathbf{u} \cdot \nabla) \mathbf{u} \cdot \mathbf{v}& - \int_{\Omega} \operatorname{div}(\mathbf{v}) p & &= \int \mathbf{f}  \cdot \mathbf{v}  && \forall \mathbf{v} \in \mathbf{V}, \\ 
- \int_{\Omega} \operatorname{div}(\mathbf{u}) q & & 
+ \int_{\Omega} \lambda q
&= 0 && \forall q \in Q, \\
& \int_{\Omega} \mu p & &= 0 && \forall \mu \in \mathbb{R}.
\end{align}



In [ ]:
mesh = Mesh (unit_square.GenerateMesh(maxh=0.05)); nu = Parameter(1)
V = VectorH1(mesh,order=3,dirichlet="bottom|right|top|left")
Q = H1(mesh,order=2); 
N = NumberSpace(mesh); 
X = V*Q*N
(u,p,lam), (v,q,mu) = X.TnT()
a = BilinearForm(X)
a += (nu*InnerProduct(grad(u),grad(v))+InnerProduct(grad(u)*u,v)
      -div(u)*q-div(v)*p-lam*q-mu*p)*dx

In [ ]:
gfu = GridFunction(X)
gfu.components[0].Set(CoefficientFunction((4*x*(1-x),0)),
                      definedon=mesh.Boundaries("top"))

In [ ]:
SimpleNewtonSolve(gfu,a)
scenep = Draw(gfu.components[1],mesh,"p")
sceneu = Draw(gfu.components[0],mesh,"u")

In [ ]:
nu.Set(0.01)
SimpleNewtonSolve(gfu,a)
sceneu.Redraw()
scenep.Redraw()

In [ ]:
nu.Set(0.001)
SimpleNewtonSolve(gfu,a)
sceneu.Redraw()
scenep.Redraw()

In [ ]:
nu.Set(0.001)
gfu.components[0].Set(CoefficientFunction((4*x*(1-x),0)),definedon=mesh.Boundaries("top"))
Newton(a,gfu,maxit=20,dampfactor=0.1)
sceneu.Redraw()
scenep.Redraw()